In [16]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import logging

In [17]:
import pandas as pd
import numpy as np
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch import optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import time

In [18]:
def cutmix(batch,alpha=1):
    inputs,labels = batch
    indices = torch.randperm(inputs.size(0))
    shuffled_inputs = inputs[indices]
    shuffled_labels = labels[indices]

    weight,height= inputs.shape[2:]
    lamb = np.random.beta(alpha, alpha)

    rx=np.random.uniform(0, weight)
    ry=np.random.uniform(0, height)

    rw=weight*np.sqrt(1-lamb)
    rh=height*np.sqrt(1-lamb)

    x0 = int(np.round(max(rx - rw / 2, 0)))
    x1 = int(np.round(min(rx + rw / 2, weight)))
    y0 = int(np.round(max(ry - rh / 2, 0)))
    y1 = int(np.round(min(ry + rh / 2, height)))

    inputs[:, :, y0:y1, x0:x1] = shuffled_inputs[:, :, y0:y1, x0:x1]
    labels = (labels, shuffled_labels, lamb)

    return inputs,labels

class CutMixCollator:
    def __init__(self, alpha):
        self.alpha = alpha

    def __call__(self, batch):
        batch = torch.utils.data.dataloader.default_collate(batch)
        batch = cutmix(batch, self.alpha)
        return batch

class CutMixCriterion:
    def __init__(self, reduction):
        self.criterion = nn.CrossEntropyLoss(reduction=reduction)

    def __call__(self, preds, targets):
        targets1, targets2, lam = targets
        return lam * self.criterion(
            preds, targets1) + (1 - lam) * self.criterion(preds, targets2)

In [19]:
def get_loader(batch_size=128,num_workers=4,cutmix=True,alpha=1):
    if cutmix==True:
        collator = CutMixCollator(alpha)
    else:
        collator = torch.utils.data.dataloader.default_collate

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    train_dataset = datasets.CIFAR10(root='cifar', train=True, transform=transforms.Compose([transforms.RandomHorizontalFlip(),
                    transforms.RandomCrop(32, 4),
                    transforms.ToTensor(),
                    normalize,]), download=True)
      
    dev_dataset=datasets.CIFAR10(root='cifar', train=False, transform=transforms.Compose([                                                                    
                transforms.ToTensor(),
                normalize,]))

    train_dataloader = torch.utils.data.DataLoader(train_dataset,      
                       batch_size=batch_size, shuffle=True,
                       num_workers=num_workers, pin_memory=True,
                       drop_last=True,collate_fn=collator)
    
    dev_dataloader = torch.utils.data.DataLoader( dev_dataset, 
                     batch_size=batch_size, shuffle=False,
                     num_workers=num_workers, pin_memory=True,
                     drop_last=False)
    return train_dataloader,dev_dataloader

In [20]:
train_dataloader,dev_dataloader=get_loader(batch_size=128,num_workers=4,cutmix=True,alpha=1)

Files already downloaded and verified


In [21]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(3,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [22]:
torch.cuda.is_available() 
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = ResNet(BasicBlock, [2,2,2,2], 10)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [23]:
criterion1 = CutMixCriterion(reduction='mean')
criterion2 = nn.CrossEntropyLoss(size_average=True)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
optimizer = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.9, weight_decay=1e-4,nesterov=True)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 300, 0)

/usr/local/miniconda3/envs/python_gpu/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [24]:
trainingloss=[]
trainingacc=[]
valloss=[]
valacc=[]
valacc5=[]

In [25]:

def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)

    return logger

In [26]:
logger = get_logger('cutmix终稿3/exp.log')
logger.info('start training!')

[2021-06-19 00:53:55,179][<ipython-input-26-b7fe465fc713>][line:2][INFO] start training!
[2021-06-19 00:53:55,179][<ipython-input-26-b7fe465fc713>][line:2][INFO] start training!


In [ ]:

epoch=300
best_accuracy=0.0
es=0
start_time=time.time()
for i in range(epoch):
    model.train()
    total_loss=0.0
    accuracy=0.0
    total_correct=0.0
    total_data_num = len(train_dataloader.dataset)
    steps = 0.0
    #训练
    for batch in train_dataloader:
        steps+=1
        optimizer.zero_grad() 
        # 取数据
        inputs, labels = batch
        inputs = inputs.to(device)
        labels, shuffled_labels, lamb=labels
        labels=(labels.to(device), shuffled_labels.to(device), lamb)
        #targets1, targets2, lam = targets
        #, labels.to(device)  # 将输入和目标在每一步都送入GPU
        outputs = model(inputs)
        #_, outputs = torch.max(outputs.data, 1)
        optimizer.zero_grad()
        loss = criterion1(outputs, labels).to(device)

        loss.backward()
        optimizer.step()  

        total_loss = total_loss + loss.item() 

        #计算正确率
        labels, shuffled_labels, lamb=labels
        correct1 = (torch.max(outputs, dim=1)[1]  #get the indices
                   .view(labels.size()) == labels).sum()
        correct2 = (torch.max(outputs, dim=1)[1]  #get the indices
                   .view(shuffled_labels.size()) == shuffled_labels).sum()

        correct = (lamb * correct1.item() + (1 - lamb) * correct2.item()) 

        total_correct = total_correct + correct

 
        if steps%100==0:
            print("Epoch %d_%.3f%%:  Training average Loss: %f"
                      %(i, steps * train_dataloader.batch_size*100/len(train_dataloader.dataset),total_loss/steps))
    logger.info("Epoch %d :  Training average Loss: %f, Training accuracy: %f%%,Total Time:%f"
      %(i, total_loss/steps, total_correct*100/total_data_num,time.time()-start_time))   
    trainingloss.append(total_loss/steps)
    trainingacc.append(total_correct/total_data_num) 
    
    
    
    
    
    
    
    
    #验证
    scheduler.step()
    model.eval()
    total_loss=0.0
    accuracy=0.0
    total_correct=0.0
    total_correctk=0.0
    total_data_num = len(dev_dataloader.dataset)
    steps = 0.0    
    for batch in dev_dataloader:
        steps+=1
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)  # 将输入和目标在每一步都送入GPU
        outputs = model(inputs)
        loss = criterion2(outputs, labels)  
        total_loss = total_loss + loss.item() 
        correct = (torch.max(outputs, dim=1)[1]  #get the indices
                   .view(labels.size()) == labels).sum()
        total_correct = total_correct + correct.item()

        maxk = max((1,5))
        yresize = labels.view(-1,1)
        _, pred = outputs.topk(maxk, 1, True, True)

        correctk = torch.eq(pred, yresize).sum()

        #correct = (torch.max(outputs, dim=1)[1]  #get the indices
                   #.view(labels.size()) == labels).sum()
        total_correctk = total_correctk + correctk.item()
        
    logger.info("Epoch %d :  Verification average Loss: %f, Verification accuracy: %f%%,Verification 5 accuracy: %f%%,Total Time:%f"
      %(i, total_loss/steps, total_correct*100/total_data_num,total_correctk*100/total_data_num,time.time()-start_time))  
    #print("Epoch %d :  Verification 5 accuracy: %f%%,Total Time:%f"
      #%(i,  total_correctk*100/total_data_num,time.time()-start_time))  
    valloss.append(total_loss/steps)
    valacc.append(total_correct/total_data_num) 
    valacc5.append(total_correctk/total_data_num) 
    if best_accuracy < total_correct/total_data_num :
        es = 0
        best_accuracy =total_correct/total_data_num 
        torch.save(model,'cutmix终稿3/epoch_%d_accuracy_%f'%(i,total_correct/total_data_num))
        logger.info('Model is saved in cutmix终稿3/epoch_%d_accuracy_%f'%(i,total_correct/total_data_num))
        #torch.cuda.empty_cache()
    #torch.save(model.state_dict(), 'cutmix/epoch_%d_accuracy_%f.pkl'%(i,total_correct/total_data_num))

Epoch 0_25.600%:  Training average Loss: 3.040181
Epoch 0_51.200%:  Training average Loss: 2.613891
Epoch 0_76.800%:  Training average Loss: 2.468011


[2021-06-19 00:54:27,998][<ipython-input-27-94dce6441585>][line:50][INFO] Epoch 0 :  Training average Loss: 2.389646, Training accuracy: 16.602943%,Total Time:24.414154
[2021-06-19 00:54:27,998][<ipython-input-27-94dce6441585>][line:50][INFO] Epoch 0 :  Training average Loss: 2.389646, Training accuracy: 16.602943%,Total Time:24.414154
[2021-06-19 00:54:29,710][<ipython-input-27-94dce6441585>][line:92][INFO] Epoch 0 :  Verification average Loss: 1.949977, Verification accuracy: 22.850000%,Verification 5 accuracy: 81.050000%,Total Time:26.126561
[2021-06-19 00:54:29,710][<ipython-input-27-94dce6441585>][line:92][INFO] Epoch 0 :  Verification average Loss: 1.949977, Verification accuracy: 22.850000%,Verification 5 accuracy: 81.050000%,Total Time:26.126561
[2021-06-19 00:54:33,656][<ipython-input-27-94dce6441585>][line:102][INFO] Model is saved in cutmix终稿3/epoch_0_accuracy_0.228500
[2021-06-19 00:54:33,656][<ipython-input-27-94dce6441585>][line:102][INFO] Model is saved in cutmix终稿3/epoc

Epoch 1_25.600%:  Training average Loss: 2.145577
Epoch 1_51.200%:  Training average Loss: 2.131984
Epoch 1_76.800%:  Training average Loss: 2.124279


[2021-06-19 00:54:59,280][<ipython-input-27-94dce6441585>][line:50][INFO] Epoch 1 :  Training average Loss: 2.103731, Training accuracy: 21.387986%,Total Time:55.696569
[2021-06-19 00:54:59,280][<ipython-input-27-94dce6441585>][line:50][INFO] Epoch 1 :  Training average Loss: 2.103731, Training accuracy: 21.387986%,Total Time:55.696569
[2021-06-19 00:55:01,956][<ipython-input-27-94dce6441585>][line:92][INFO] Epoch 1 :  Verification average Loss: 1.764314, Verification accuracy: 33.980000%,Verification 5 accuracy: 87.820000%,Total Time:58.372976
[2021-06-19 00:55:01,956][<ipython-input-27-94dce6441585>][line:92][INFO] Epoch 1 :  Verification average Loss: 1.764314, Verification accuracy: 33.980000%,Verification 5 accuracy: 87.820000%,Total Time:58.372976
[2021-06-19 00:55:08,568][<ipython-input-27-94dce6441585>][line:102][INFO] Model is saved in cutmix终稿3/epoch_1_accuracy_0.339800
[2021-06-19 00:55:08,568][<ipython-input-27-94dce6441585>][line:102][INFO] Model is saved in cutmix终稿3/epoc

Epoch 2_25.600%:  Training average Loss: 2.054627
Epoch 2_51.200%:  Training average Loss: 2.050564
Epoch 2_76.800%:  Training average Loss: 2.045203


...

Epoch 3_25.600%:  Training average Loss: 2.007472
Epoch 3_51.200%:  Training average Loss: 1.996073
Epoch 3_76.800%:  Training average Loss: 1.995265


[2021-06-19 00:56:02,857][<ipython-input-27-94dce6441585>][line:50][INFO] Epoch 3 :  Training average Loss: 1.969253, Training accuracy: 28.746775%,Total Time:119.273791
[2021-06-19 00:56:02,857][<ipython-input-27-94dce6441585>][line:50][INFO] Epoch 3 :  Training average Loss: 1.969253, Training accuracy: 28.746775%,Total Time:119.273791
[2021-06-19 00:56:04,509][<ipython-input-27-94dce6441585>][line:92][INFO] Epoch 3 :  Verification average Loss: 1.493865, Verification accuracy: 43.640000%,Verification 5 accuracy: 92.560000%,Total Time:120.925516
[2021-06-19 00:56:04,509][<ipython-input-27-94dce6441585>][line:92][INFO] Epoch 3 :  Verification average Loss: 1.493865, Verification accuracy: 43.640000%,Verification 5 accuracy: 92.560000%,Total Time:120.925516
[2021-06-19 00:56:08,467][<ipython-input-27-94dce6441585>][line:102][INFO] Model is saved in cutmix终稿3/epoch_3_accuracy_0.436400
[2021-06-19 00:56:08,467][<ipython-input-27-94dce6441585>][line:102][INFO] Model is saved in cutmix终稿3/

Epoch 4_25.600%:  Training average Loss: 1.958737
Epoch 4_51.200%:  Training average Loss: 1.952347
Epoch 4_76.800%:  Training average Loss: 1.947876


[2021-06-19 00:56:33,052][<ipython-input-27-94dce6441585>][line:50][INFO] Epoch 4 :  Training average Loss: 1.916666, Training accuracy: 31.616069%,Total Time:149.469056
[2021-06-19 00:56:33,052][<ipython-input-27-94dce6441585>][line:50][INFO] Epoch 4 :  Training average Loss: 1.916666, Training accuracy: 31.616069%,Total Time:149.469056
[2021-06-19 00:56:34,707][<ipython-input-27-94dce6441585>][line:92][INFO] Epoch 4 :  Verification average Loss: 1.415240, Verification accuracy: 47.210000%,Verification 5 accuracy: 93.450000%,Total Time:151.123877
[2021-06-19 00:56:34,707][<ipython-input-27-94dce6441585>][line:92][INFO] Epoch 4 :  Verification average Loss: 1.415240, Verification accuracy: 47.210000%,Verification 5 accuracy: 93.450000%,Total Time:151.123877
[2021-06-19 00:56:38,653][<ipython-input-27-94dce6441585>][line:102][INFO] Model is saved in cutmix终稿3/epoch_4_accuracy_0.472100
[2021-06-19 00:56:38,653][<ipython-input-27-94dce6441585>][line:102][INFO] Model is saved in cutmix终稿3/

Epoch 5_25.600%:  Training average Loss: 1.894223


In [ ]:
trainingloss=pd.DataFrame(data=trainingloss)
trainingloss.to_csv('cutmix终稿3/trainingloss.csv',encoding='utf-8')

trainingacc=pd.DataFrame(data=trainingacc)
trainingacc.to_csv('cutmix终稿3/trainingacc.csv',encoding='utf-8')

valloss=pd.DataFrame(data=valloss)
valloss.to_csv('cutmix终稿3/valloss.csv',encoding='utf-8')

valacc=pd.DataFrame(data=valacc)
valacc.to_csv('cutmix终稿3/valacc.csv',encoding='utf-8')

valacc5=pd.DataFrame(data=valacc5)
valacc5.to_csv('cutmix终稿3/valacc5.csv',encoding='utf-8')

In [15]:
import os
os.mkdir('cutmix终稿3')